Installing the required packages

In [ ]:
%%capture
import torch

# 1. Install Unsloth (Optimized for Colab)
# We allow unsloth to handle the dependency resolution for Colab
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# 2. Install/Upgrade Xformers and TRL
# We force these upgrades to ensure compatibility with the T4 GPU on Colab
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

# 3. Verify Pytorch & GPU
print(f"Pytorch Version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: No GPU detected. Please check Runtime settings.")

Loading the fine-tuned model from the drive

In [ ]:
from unsloth import FastLanguageModel
from google.colab import drive
import torch
import os

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Define Path to your SAVED FINAL MODEL
# (This matches the path we saved to earlier)
model_path = "/content/drive/MyDrive/DocSpider_Mistral_Final"

print(f"🔄 Loading model from: {model_path} ...")

# 3. Load the Model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_path,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

print("✅ Model loaded successfully! Ready for testing.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Mounted at /content/drive
🔄 Loading model from: /content/drive/MyDrive/DocSpider_Mistral_Final ...
==((====))==  Unsloth 2026.1.4: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

Unsloth 2026.1.4 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


✅ Model loaded successfully! Ready for testing.


Getting the test instances (predictions) for the test data from dev.json

In [ ]:
import json
from tqdm import tqdm
import os

# --- PATHS ---
output_file = "/content/drive/MyDrive/docspider_predictions.json" # <--- Account 1's copy
dev_file_path = "/content/dev.json"
tables_file_path = "/content/collections.json"
save_every = 10
# -------------

# 1. Load Schemas
schema_map = {}
if os.path.exists(tables_file_path):
    print(f"✅ Loading schemas from: {tables_file_path}")
    with open(tables_file_path, 'r') as f:
        tables_data = json.load(f)
        for db in tables_data:
            schema_map[db['db_id']] = db.get('schema', db)
else:
    print(f"⚠️ WARNING: Could not find {tables_file_path}. Model might fail to see field names!")

# 2. Load Questions
if not os.path.exists(dev_file_path):
    raise FileNotFoundError(f"❌ ERROR: Please upload {dev_file_path} to Colab files!")

print(f"✅ Loading questions from: {dev_file_path}")
with open(dev_file_path, 'r') as f:
    test_data = json.load(f)

# --- AUTO-RESUME LOGIC ---
start_index = 0
generated_results = []

if os.path.exists(output_file):
    try:
        with open(output_file, 'r') as f:
            generated_results = json.load(f)
            start_index = len(generated_results)
            print(f"🔄 Found progress in Drive! Resuming from question #{start_index}...")
    except:
        print("⚠️ Output file exists but unreadable. Starting fresh.")
else:
    print("🆕 No previous progress found in Account 1's Drive. Starting from scratch.")

print(f"🚀 Starting generation for {len(test_data) - start_index} remaining examples...")

# 3. Main Loop
for i, entry in enumerate(tqdm(test_data[start_index:])):
    current_idx = start_index + i

    question = entry.get('question', '')
    db_id = entry.get('db_id', '')
    q_id = entry.get('question_id', 'N/A')

    # Get Schema
    schema_info = entry.get('schema', entry.get('struct_in', ''))
    if not schema_info and db_id in schema_map:
        schema_info = schema_map[db_id]

    schema_str = json.dumps(schema_info, indent=2) if isinstance(schema_info, dict) else str(schema_info)

    # Prompt
    prompt = f"""<s>[INST] Schema: {schema_str}
Question: {question}
Task: Provide the reasoning and then the NoSQL query. [/INST]"""

    # Generate
    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens = 512,
        use_cache = True,
    )

    full_text = tokenizer.batch_decode(outputs)[0]

    # Extract
    reasoning = ""
    mql_query = "ERROR_PARSING"

    try:
        response_part = full_text.split("[/INST]")[-1]
        if "[MQL]" in response_part:
            parts = response_part.split("[MQL]")
            reasoning = parts[0].strip()
            mql_query = parts[1].replace("</s>", "").strip()
        else:
            mql_query = response_part.replace("</s>", "").strip()
    except:
        mql_query = "ERROR_PARSING"

    generated_results.append({
        "question_id": q_id,
        "question": question,
        "gold_mql": entry.get('query', ''),
        "generated_mql": mql_query,
        "reasoning": reasoning,
        "db_id": db_id
    })

    # Save to Drive
    if len(generated_results) % save_every == 0:
        with open(output_file, "w") as f:
            json.dump(generated_results, f, indent=2)

# Final Save
with open(output_file, "w") as f:
    json.dump(generated_results, f, indent=2)

print(f"\n✅ SUCCESS! All results saved to: {output_file}")

✅ Loading schemas from: /content/collections.json
✅ Loading questions from: /content/dev.json
🔄 Found progress in Drive! Resuming from question #330...
🚀 Starting generation for 290 remaining examples...


100%|██████████| 290/290 [1:28:51<00:00, 18.38s/it]


✅ SUCCESS! All results saved to: /content/drive/MyDrive/docspider_predictions.json
